<a href="https://colab.research.google.com/github/gg2001/transformer-circuits/blob/master/puzzles/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Mech Interp Puzzle 1: Suspiciously Similar Embeddings in GPT-Neo](https://www.alignmentforum.org/posts/eLNo7b56kQQerCzp2/mech-interp-puzzle-1-suspiciously-similar-embeddings-in-gpt)

In [1]:
%pip install transformer_lens plotly nbformat

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/

In [2]:
from tqdm import tqdm
from typing import Callable
from transformer_lens import HookedTransformer
import plotly.express as px
import pandas as pd
import torch
import heapq

In [3]:
model = HookedTransformer.from_pretrained("gpt-neo-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Loaded pretrained model gpt-neo-small into HookedTransformer


In [4]:
W_E = model.W_E
W_E.requires_grad = False

In [5]:
# subsample = torch.randperm(model.cfg.d_vocab)[:5000].to(model.cfg.device)
# W_E = model.W_E[subsample]  # Take a random subset of 5,000 for memory reasons

In [6]:
W_E_normed = W_E / W_E.norm(dim=-1, keepdim=True) # [d_vocab, d_model]
# cosine_sims = W_E_normed @ W_E_normed.T # [d_vocab, d_vocab]

In [7]:
# px.histogram(
#    cosine_sims.flatten().detach().cpu().numpy(),
#    title="Pairwise cosine sims of embedding",
#)

In [8]:
def get_cosine_sim(token0: str, token1: str) -> float:
    token0_ids = model.tokenizer.encode(token0)
    token1_ids = model.tokenizer.encode(token1)

    if len(token0_ids) != 1:
        raise ValueError(f"Token {token0} is not a single token")
    if len(token1_ids) != 1:
        raise ValueError(f"Token {token1} is not a single token")

    return (W_E_normed[token0_ids[0]] @ W_E_normed[token1_ids[0]]).item()

In [9]:
get_cosine_sim("+", "plus")

0.9437829256057739

In [10]:
TokenHeap = list[tuple[float, int, int, str, str]]
num_tokens = W_E_normed.size(0)

def push_token_heap(
    heap: TokenHeap,
    values: list[float],
    token0: list[int],
    token1: list[int],
    compare: Callable[[float, float], bool],
    top_k: int
):
    for value, t0, t1 in zip(values, token0, token1):
        if len(heap) < top_k:
            heapq.heappush(heap, (value, t0, t1))
        else:
            if compare(value, heap[0][0]):
                heapq.heappushpop(heap, (value, t0, t1))

In [11]:
def get_cosine_sims(token0) -> TokenHeap:
    token0_ids = model.tokenizer.encode(token0)
    if len(token0_ids) != 1:
        raise ValueError(f"Token {token0} is not a single token")

    token_cosine_sims = W_E_normed[token0_ids[0]] @ W_E_normed.T
    heap = []
    for token1_id in range(token_cosine_sims.size(0)):
        value = token_cosine_sims[token1_id].item()
        heap.append((value, token0_ids[0], token1_id))

    for i in range(len(heap)):
        heap[i] += (model.tokenizer.convert_ids_to_tokens(heap[i][1]), model.tokenizer.convert_ids_to_tokens(heap[i][2]))

    heap.sort(key=lambda x: x[0], reverse=True)

    return heap

In [12]:
get_cosine_sims("+")[:10]

[(1.0000001192092896, 10, 10, '+', '+'),
 (0.9724709987640381, 10, 1343, '+', 'Ġ+'),
 (0.9491391777992249, 10, 28200, '+', '+,'),
 (0.9476434588432312, 10, 5556, '+', 'Ġplus'),
 (0.947392463684082, 10, 27371, '+', '349'),
 (0.9462127685546875, 10, 31714, '+', '479'),
 (0.9461718201637268, 10, 33459, '+', '459'),
 (0.9461280107498169, 10, 33300, '+', '649'),
 (0.9461238384246826, 10, 22996, '+', '307'),
 (0.9461236000061035, 10, 31911, '+', '449')]

In [13]:
get_cosine_sims("+")[num_tokens-10:]

[(0.7769637107849121, 10, 36914, '+', 'ĠFraz'),
 (0.7748276591300964, 10, 38245, '+', 'ĠByrne'),
 (0.773963987827301, 10, 46760, '+', 'ĠBrowne'),
 (0.7715131044387817, 10, 41807, '+', 'leanor'),
 (0.7701801657676697, 10, 35763, '+', 'ĠHirosh'),
 (0.7675925493240356, 10, 47005, '+', 'ĠSweeney'),
 (0.7590717673301697, 10, 38802, '+', 'ĠKarin'),
 (0.7514899373054504, 10, 40627, '+', 'ĠMakoto'),
 (0.7448464632034302, 10, 33891, '+', 'ĠShinji'),
 (0.7336776852607727, 10, 34926, '+', 'ĠGors')]

In [14]:
batch = 1000
top_k = 1000
top_heap: TokenHeap = []
bottom_heap: TokenHeap = []
sum_cosine_sims = 0.0
count_cosine_sims = 0


for i in tqdm(range(0, num_tokens, batch)):
    # Get the cosine similarities of the batch
    token_cosine_sims = (W_E_normed[i:i+batch] @ W_E_normed[i + 1:].T)

    # Get the top diagonal of the batch
    mask = torch.tril(torch.ones(token_cosine_sims.size(0), token_cosine_sims.size(1), dtype=torch.bool, device=token_cosine_sims.device), diagonal=-1)
    cosine_sims_top = token_cosine_sims.masked_fill(mask, float('-inf'))
    flattened = cosine_sims_top.view(-1)

    # Incrementally calculate the mean
    valid_mask = flattened != float('-inf')
    sum_cosine_sims += flattened[valid_mask].sum().item()
    count_cosine_sims += valid_mask.sum().item()

    # Get the highest top_k cosine similarities
    top_cosine_sims = torch.topk(flattened, top_k)
    top_values = top_cosine_sims.values.tolist()
    top_token0 = ((top_cosine_sims.indices // (num_tokens - i - 1)) + i).tolist()
    top_token1 = ((top_cosine_sims.indices % (num_tokens - i - 1)) + i + 1).tolist()
    push_token_heap(top_heap, top_values, top_token0, top_token1, lambda x, y: x > y, top_k)

    # Get the lowest top_k cosine similarities
    flattened[valid_mask] *= -1
    bottom_cosine_sims = torch.topk(flattened, top_k)
    bottom_values = (bottom_cosine_sims.values.clone() * -1).tolist()
    bottom_token0 = ((bottom_cosine_sims.indices // (num_tokens - i - 1)) + i).tolist()
    bottom_token1 = ((bottom_cosine_sims.indices % (num_tokens - i - 1)) + i + 1).tolist()
    push_token_heap(bottom_heap, bottom_values, bottom_token0, bottom_token1, lambda x, y: x < y, top_k)


# Sort heaps
top_heap.sort(key=lambda x: x[0], reverse=True)
bottom_heap.sort(key=lambda x: x[0], reverse=False)

# Get the token strings
for i in range(top_k):
    top_heap[i] += (model.tokenizer.convert_ids_to_tokens(top_heap[i][1]), model.tokenizer.convert_ids_to_tokens(top_heap[i][2]))
    bottom_heap[i] += (model.tokenizer.convert_ids_to_tokens(bottom_heap[i][1]), model.tokenizer.convert_ids_to_tokens(bottom_heap[i][2]))

# Compute mean
mean = sum_cosine_sims / count_cosine_sims

print()
print(f"Mean cosine similarity: {mean}")
print(f"Highest cosine similarity: {top_heap[0][0]} (token0={top_heap[0][3]}, token1={top_heap[0][4]})")
print(f"Lowest cosine similarity: {bottom_heap[0][0]} (token0={bottom_heap[0][3]}, token1={bottom_heap[0][4]})")

100%|██████████| 51/51 [00:04<00:00, 11.60it/s]


Mean cosine similarity: 0.9293529306070797
Highest cosine similarity: 0.9999352097511292 (token0=TPPStreamerBot, token1=EStreamFrame)
Lowest cosine similarity: 0.6320329904556274 (token0=ĠC, token1=ĠShinji)


In [15]:
# Add new heap to store token pairs closest to the mean
mean_heap: TokenHeap = []

for i in tqdm(range(0, num_tokens, batch)):
    # Get the cosine similarities of the batch
    token_cosine_sims = (W_E_normed[i:i + batch] @ W_E_normed[i + 1:].T)

    # Get the top diagonal of the batch
    mask = torch.tril(torch.ones(token_cosine_sims.size(0), token_cosine_sims.size(1), dtype=torch.bool, device=token_cosine_sims.device), diagonal=-1)
    cosine_sims_top = token_cosine_sims.masked_fill(mask, float('-inf'))
    flattened = cosine_sims_top.view(-1)

    # Compute the absolute differences from the mean
    valid_mask = flattened != float('-inf')
    valid_cosine_sims = flattened[valid_mask]
    differences_from_mean = torch.abs(valid_cosine_sims - mean)

    # Get the top_k cosine similarities closest to the mean
    closest_cosine_sims = torch.topk(-differences_from_mean, top_k)  # Negative because we want the smallest differences
    closest_values = valid_cosine_sims[closest_cosine_sims.indices].tolist()
    closest_token0 = ((closest_cosine_sims.indices // (num_tokens - i - 1)) + i).tolist()
    closest_token1 = ((closest_cosine_sims.indices % (num_tokens - i - 1)) + i + 1).tolist()
    push_token_heap(mean_heap, closest_values, closest_token0, closest_token1, lambda x, y: abs(x - mean) < abs(y - mean), top_k)


# Sort heap
mean_heap.sort(key=lambda x: abs(x[0] - mean))

# Get the token strings for the pairs closest to the mean
for i in range(top_k):
    mean_heap[i] += (model.tokenizer.convert_ids_to_tokens(mean_heap[i][1]), model.tokenizer.convert_ids_to_tokens(mean_heap[i][2]))

print()
print(f"Closest cosine similarity to the mean: {mean_heap[0][0]} (token0={mean_heap[0][3]}, token1={mean_heap[0][4]})")

100%|██████████| 51/51 [00:01<00:00, 45.53it/s]


Closest cosine similarity to the mean: 0.9293529391288757 (token0=Q, token1=ĠWheat)


In [16]:
columns = ["cosine_sum", "token0_id", "token1_id", "token0_str", "token1_str"]
top_df = pd.DataFrame(top_heap, columns=columns)
bottom_df = pd.DataFrame(bottom_heap, columns=columns)
mean_df = pd.DataFrame(mean_heap, columns=columns)

In [17]:
top_df.head(n=5)

,cosine_sum,token0_id,token1_id,token0_str,token1_str
0,0.999935,37579,43177,TPPStreamerBot,EStreamFrame
1,0.999935,37574,43177,StreamerBot,EStreamFrame
2,0.999934,28666,43177,PsyNetMessage,EStreamFrame
3,0.999934,187,36173,ÿ,ĠRandomRedditor
4,0.999933,178,187,ö,ÿ


In [18]:
top_token_counts = pd.concat([top_df['token0_str'], top_df['token1_str']]).value_counts()
top_token_counts.head(n=15)

,count
ÿ,51
EStreamFrame,51
PsyNetMessage,49
ù,49
TPPStreamerBot,49
Á,49
ĠRandomRedditor,48
cloneembedreportprint,47
StreamerBot,47
ý,47


In [19]:
bottom_df.head(n=5)

,cosine_sum,token0_id,token1_id,token0_str,token1_str
0,0.632033,327,33891,ĠC,ĠShinji
1,0.636070,327,34926,ĠC,ĠGors
2,0.641810,564,34926,ĠâĢ,ĠGors
3,0.641955,406,33891,ĠL,ĠShinji
4,0.642201,327,40627,ĠC,ĠMakoto


In [20]:
bottom_token_counts = pd.concat([bottom_df['token0_str'], bottom_df['token1_str']]).value_counts()
bottom_token_counts.head(n=15)

,count
ĠC,94
ĠGors,89
ĠShinji,79
ĠMakoto,70
ĠâĢ,57
ĠP,57
ĠL,45
ĠM,41
ĠD,40
ĠKarin,39


In [21]:
mean_df.head(n=5)

,cosine_sum,token0_id,token1_id,token0_str,token1_str
0,0.929353,48,34744,Q,ĠWheat
1,0.929353,51,8211,T,ĠPacific
2,0.929353,89,15880,z,usions
3,0.929353,104,13420,«,ĠVenezuel
4,0.929353,118,16580,º,NOTE


In [22]:
mean_token_counts = pd.concat([mean_df['token0_str'], mean_df['token1_str']]).value_counts()
mean_token_counts.head(n=15)

,count
ose,5
×,4
Ã,4
fore,4
ĠN,4
Ġke,3
N,3
reat,3
ime,3
|,3
